In [3]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from lz import * 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
sys.path.insert(0,'/data1/xinglu/prj/open-reid')
from train import * 
from torch import autograd 
from torch.nn import  functional as F 
import torchvision

In [33]:
init_dev((3,))
dataset, num_classes, train_loader, val_loader, test_loader = get_data(
    'cuhk03',0,'/home/xinglu/.torch/data',256,128,8,4,4,True)

model=models.create('attention50')
# checkpoint=load_checkpoint('./logs.tri.bak/model_best.pth.tar')
# model.load_state_dict(checkpoint['state_dict'])
# model=nn.DataParallel(model).cuda()

In [9]:
lz.mkdir_p('graph',delete=True)

In [10]:
writer=SummaryWriter('graph/resnet')
res=model(torch.autograd.Variable(torch.Tensor(1,3,256,128),requires_grad=True)) 
res.size()

writer.add_graph(model,res)
writer.close()

torch.Size([1, 2048, 8, 4])

In [13]:
writer=SummaryWriter('graph/resnet.base')
res=model.base(torch.autograd.Variable(torch.Tensor(1,3,256,256),requires_grad=True)) 
res.size()

writer.add_graph(model.base,res)
writer.close()

torch.Size([1, 1000])

In [6]:
model = torchvision.models.densenet121()

In [29]:
import torch
import torch.nn as nn
import torchvision.utils as vutils
import numpy as np
import torch.nn.functional as F
import torchvision.models as models
from tensorboardX import SummaryWriter

class Mnist(nn.Module):
    def __init__(self):
        super(Mnist, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.bn = nn.BatchNorm2d(20)
    def forward(self, x):
        x = F.max_pool2d(self.conv1(x), 2)
        x = F.relu(x)+F.relu(-x)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.bn(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.log_softmax(x)
        return x

model = Mnist()

# if you want to show the input tensor, set requires_grad=True
res = model(torch.autograd.Variable(torch.Tensor(1,1,28,28), requires_grad=True))

writer = SummaryWriter('graph/mnist')
writer.add_graph(model, res)

writer.close()

In [7]:
test_transformer = T.Compose([
    T.RectScale(256,128),
    T.ToTensor(),
])

test_loader2 = DataLoader(
    Preprocessor(list(set(dataset.query) | set(dataset.gallery)),
                 root=dataset.images_dir, transform=test_transformer),
    batch_size=8, num_workers=2,
    shuffle=False, pin_memory=True)



In [45]:
metric = DistanceMetric(algorithm='euclidean')
# Evaluator
evaluator = Evaluator(model)
evaluator.evaluate(test_loader, dataset.query, dataset.gallery, metric)


0.65755181347148428

In [8]:
metric = DistanceMetric(algorithm='euclidean')
# Evaluator
evaluator = Evaluator(model)
acc = evaluator.evaluate(val_loader, dataset.val, dataset.val, metric,return_all=True)
print acc['cuhk03'][0]

In [29]:
8*len(test_loader),len(train_loader),len(val_loader)

(1936, 683, 242)

In [144]:
im.size()
t=nn.AvgPool2d((4,4))(im)
type(t),type(im)

torch.Size([3, 256, 128])

(torch.autograd.variable.Variable, torch.FloatTensor)

True

In [17]:
if 'writer' in globals().keys():
    writer.close()
import lz
lz.mkdir_p('./tri.tsne',delete=True)
writer=SummaryWriter('./tri.tsne')

In [18]:
features=collections.OrderedDict() 
labels=collections.OrderedDict()
images=collections.OrderedDict()
for (imgs,fnames,pids,cids), (imgs_ori,_,_,_) in zip(test_loader,test_loader2):
#     imgs=torch.from_numpy((imgs)) 
    imgs=torch.autograd.Variable(imgs,volatile=True)
    feas=model(imgs)
#     imgs.size(),feas.size()
    for name,f,l,im in zip(fnames, feas,pids,imgs_ori):
        features[name] =f 
        labels[name]=l
        images[name]=nn.AvgPool2d((4,4))(im).data
#     if len(features) > 500:
#         break 

images.values()[0].size()
features.values()[0].size( )


labels.values()[0]

images=torch.stack(images.values()).float()
# images=torch.mean(images,1)
features = torch.stack(features.values())
features.size()
labels = np.array(labels.values()) 
labels = torch.from_numpy(labels) 
# labels = labels.view((-1,1))
labels = torch.autograd.Variable(labels)

torch.Size([3, 64, 32])

torch.Size([128])

1284

torch.Size([1930, 128])

In [19]:
features.size(),labels.size(),images.size()

(torch.Size([1930, 128]), torch.Size([1930]), torch.Size([1930, 3, 64, 32]))

In [20]:
writer.add_embedding(features.data, metadata=labels.data, label_img=images,global_step=0,tag='test.spirit')        

writer.add_embedding(features.data, metadata=labels.data ,global_step=1,tag='test.label')  

In [156]:
writer.close()

In [119]:
im=images[0].numpy().transpose((1,2,0))
im.shape

im.max(),im.min(),

plt.imshow(im)

(256, 128, 3)

In [22]:

test_transformer = T.Compose([
    T.RectScale(256,128),
    T.ToTensor(),
])

val_loader2 = DataLoader(
    Preprocessor( dataset.val  ,
                 root=dataset.images_dir, transform=test_transformer),
    batch_size=8, num_workers=2,
    shuffle=False, pin_memory=True)


features=collections.OrderedDict() 
labels=collections.OrderedDict()
images=collections.OrderedDict()
for (imgs,fnames,pids,cids), (imgs_ori,_,_,_) in zip(val_loader,val_loader2):
#     imgs=torch.from_numpy((imgs)) 
    imgs=torch.autograd.Variable(imgs,volatile=True)
    feas=model(imgs)
#     imgs.size(),feas.size()
    for name,f,l,im in zip(fnames, feas,pids,imgs_ori):
        features[name] =f 
        labels[name]=l
        images[name]=nn.AvgPool2d((4,4))(im).data
#     if len(features) > 500:
#         break 

images.values()[0].size()
features.values()[0].size( )


labels.values()[0]

images=torch.stack(images.values()).float()
# images=torch.mean(images,1)
features = torch.stack(features.values())
features.size()
labels = np.array(labels.values()) 
labels = torch.from_numpy(labels) 
# labels = labels.view((-1,1))
labels = torch.autograd.Variable(labels)

torch.Size([3, 64, 32])

torch.Size([128])

0

torch.Size([1945, 128])

In [23]:
writer.add_embedding(features.data, metadata=labels.data, label_img=images,global_step=3,tag='train.spirit')        

writer.add_embedding(features.data, metadata=labels.data ,global_step=4,tag='train.label')  

In [24]:
writer.close()